In [1]:
# Import libraries
import pandas as pd
import numpy as np

anime dataset

In [2]:
anime_df = pd.read_csv("cleaned_anime.csv")
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64.0,9.26,793665
1,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.25,114262
2,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24.0,9.17,673572
3,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.16,151266
4,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10.0,9.15,93351


In [3]:
# Remove information we don't need
anime_df = anime_df.loc[:, ["anime_id", "name", "rating"]]
anime_df.head()

,anime_id,name,rating
0,5114,Fullmetal Alchemist: Brotherhood,9.26
1,28977,Gintama°,9.25
2,9253,Steins;Gate,9.17
3,9969,Gintama&#039;,9.16
4,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,9.15


In [4]:
rating_df = pd.read_csv("cleaned_rating.csv")


In [5]:
rating_df.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [6]:
# Remove missing values
rating_df.dropna(inplace=True)
# How many missing values do we have?
rating_df.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [8]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162993 sha256=4ed9195af671adc4119bdbe367385caa66b25e8090a589708c71a4451d1f1855
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [9]:
# Import the surprise library
from surprise import Reader
from surprise import Dataset

In [10]:
%%time
# Import the surprise library
from surprise import Reader
from surprise import Dataset

# reader parses the file containing the ratings
# Our rating scale is from 1 to 10 inclusive
reader = Reader(rating_scale=(1, 10))

# Load the dataframe into the model's dataset
data = Dataset.load_from_df(rating_df[["user_id", "anime_id", "rating"]], reader)
data

CPU times: user 820 ms, sys: 96.9 ms, total: 917 ms
Wall time: 919 ms


Import SVD algorithm

In [11]:
# Import the SVD algorithm
from surprise import SVD

In [12]:
# Import GridSearchCV to tune parameters
from surprise.model_selection import GridSearchCV

In [13]:
%%time

# Create parameters combinations
params = {
    "n_epochs": [10, 15], "lr_all": [0.003, 0.005, 0.007], "reg_all": [0.01, 0.02, 0.03]
}

# Run the grid search using SVD and the parameters to find the best parameters for Root Mean Square Error and Mean Absolute Error
gs = GridSearchCV(SVD, params, measures=['rmse', 'mae'], cv=3, joblib_verbose=2, n_jobs=-2)

gs.fit(data)

[Parallel(n_jobs=-2)]: Done  40 tasks      | elapsed:  8.0min


CPU times: user 11min 10s, sys: 8.43 s, total: 11min 19s
Wall time: 11min 18s


In [14]:
# Best RMSE score
print(gs.best_score["rmse"])

# Best combination of parameters for the best RMSE
print(gs.best_params["rmse"])

1.1896096576819077
{'n_epochs': 15, 'lr_all': 0.005, 'reg_all': 0.03}


In [15]:
%%time
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

CPU times: user 11.3 s, sys: 26.7 ms, total: 11.3 s
Wall time: 11.4 s


testing

In [16]:
# Just testing out the prediction method on user 4271 on anime id 7088
pred = algo.predict(4271, 7088).est
pred

8.021949776164728

In [17]:
# Import the dump function
from surprise.dump import dump
# Create the Pickle file for the SVD algorithm
dump("recommender.pkl", algo)